In [17]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import random

# === Load Data ===
file_path = "attrition.xlsx"  # Replace with your local file path
excel_file = pd.ExcelFile(file_path)
df = excel_file.parse(excel_file.sheet_names[0])

df.head()

,Attrition,Business Travel,CF_age band,CF_attrition label,Department,Education Field,emp no,Employee Number,Gender,Job Role,...,Performance Rating,Relationship Satisfaction,Standard Hours,Stock Option Level,Total Working Years,Work Life Balance,Years At Company,Years In Current Role,Years Since Last Promotion,Years With Curr Manager
0,Yes,Travel_Rarely,35 - 44,Ex-Employees,Sales,Life Sciences,STAFF-1,1,Female,Sales Executive,...,3,1,80,0,8,1,6,4,0,5
1,No,Travel_Frequently,45 - 54,Current Employees,R&D,Life Sciences,STAFF-2,2,Male,Research Scientist,...,4,4,80,1,10,3,10,7,1,7
2,Yes,Travel_Rarely,35 - 44,Ex-Employees,R&D,Other,STAFF-4,4,Male,Laboratory Technician,...,3,2,80,0,7,3,0,0,0,0
3,No,Travel_Frequently,25 - 34,Current Employees,R&D,Life Sciences,STAFF-5,5,Female,Research Scientist,...,3,3,80,0,8,3,8,7,3,0
4,No,Travel_Rarely,25 - 34,Current Employees,R&D,Medical,STAFF-7,7,Male,Laboratory Technician,...,3,4,80,1,6,3,2,2,2,2


In [18]:
# === Set random seed for reproducibility ===
np.random.seed(42)

# === Define States and Cities ===
states_cities = {
    'New York': ['New York City', 'Buffalo', 'Rochester'],
    'Virginia': ['Virginia Beach', 'Norfolk', 'Richmond'],
    'West Virginia': ['Charleston', 'Huntington', 'Morgantown'],
    'Illinois': ['Chicago', 'Aurora', 'Naperville'],
    'Pennsylvania': ['Philadelphia', 'Pittsburgh', 'Allentown'],
    'Ohio': ['Columbus', 'Cleveland', 'Cincinnati'],
    'North Carolina': ['Charlotte', 'Raleigh', 'Greensboro'],
    'Michigan': ['Detroit', 'Grand Rapids', 'Warren']
}

# Keep your original state probability
states = list(states_cities.keys())
state_prob = [0.40, 0.03, 0.01, 0.14, 0.20, 0.10, 0.07, 0.05]  # sums to 1.0

# === Assign Random States and Cities ===
num_records = len(df)
assigned_states = np.random.choice(states, size=num_records, p=state_prob)
assigned_cities = [np.random.choice(states_cities[state]) for state in assigned_states]

df['State'] = assigned_states
df['City'] = assigned_cities

# === Generate Hire Date and Last Working Date ===
min_end_date = datetime(2021, 1, 1)   # Jan 2021
max_end_date = datetime(2024, 12, 31)  # Dec 2024

def generate_dates(row):
    years = int(row['Years At Company']) if not pd.isnull(row['Years At Company']) else 0
    days_worked = max(365 * years, 90)  # Ensure at least 3 months
    
    if row['Attrition'] == 'Yes':
        # Random termination date between 2021 and 2024
        delta_days = (max_end_date - min_end_date).days
        end_date = min_end_date + timedelta(days=random.randint(0, delta_days))
        start_date = end_date - timedelta(days=days_worked)
        return pd.Series([start_date.date(), end_date.date()])
    else:
        # Active employees → set hire date based on tenure
        latest_end_date = max_end_date
        start_date = latest_end_date - timedelta(days=days_worked)
        return pd.Series([start_date.date(), None])

df[['Hire Date', 'Last Working Date']] = df.apply(generate_dates, axis=1)

# === Define Quarterly Base Attrition Probabilities (zig-zag) ===
quarterly_attrition_prob = {
    2021: {1: 0.20, 2: 0.22, 3: 0.18, 4: 0.21},
    2022: {1: 0.19, 2: 0.20, 3: 0.16, 4: 0.18},
    2023: {1: 0.15, 2: 0.14, 3: 0.12, 4: 0.13},
    2024: {1: 0.13, 2: 0.12, 3: 0.10, 4: 0.11}
}

# === State-Level Economic Adjustment ===
state_attrition_adj = {
    'New York': -0.03,
    'Illinois': -0.02,
    'Pennsylvania': -0.01,
    'Virginia': 0.00,
    'Ohio': +0.01,
    'North Carolina': +0.02,
    'Michigan': +0.03,
    'West Virginia': +0.04
}

# === Tenure Adjustment Function ===
def get_tenure_adjustment(tenure):
    if tenure < 1:
        return 0.08
    elif tenure <= 3:
        return 0.04
    elif tenure <= 5:
        return 0.0
    else:
        return -0.05

# === Assign Attrition Probability ===
def assign_attrition_prob(row):
    hire_date = pd.to_datetime(row['Hire Date'])
    year = hire_date.year
    quarter = (hire_date.month - 1) // 3 + 1

    base_prob = quarterly_attrition_prob.get(year, {}).get(quarter, 0.12)
    tenure_adj = get_tenure_adjustment(row['Years At Company'])
    state_adj = state_attrition_adj.get(row['State'], 0)

    prob = base_prob + tenure_adj + state_adj
    return min(max(prob, 0), 1)  # clamp 0–1

df['Attrition_Prob'] = df.apply(assign_attrition_prob, axis=1)
df['Attrition_Flag'] = np.random.binomial(1, df['Attrition_Prob'])

# === Save Updated Dataset ===
output_file_path = "attrition_data.xlsx"
with pd.ExcelWriter(output_file_path, engine="openpyxl") as writer:
    df.to_excel(writer, index=False, sheet_name="HR data")

print("Updated dataset saved as:", output_file_path, "with worksheet 'HR data'")

Updated dataset saved as: attrition_data.xlsx with worksheet 'HR data'


In [19]:
# Display updated columns
df[['emp no', 'Attrition', 'Years At Company', 'Hire Date', 'Last Working Date', 'State', 'City']].head()

,emp no,Attrition,Years At Company,Hire Date,Last Working Date,State,City
0,STAFF-1,Yes,6,2015-11-28,2021-11-26,New York,New York City
1,STAFF-2,No,10,2015-01-03,None,Michigan,Warren
2,STAFF-4,Yes,0,2024-05-18,2024-08-16,Pennsylvania,Philadelphia
3,STAFF-5,No,8,2017-01-02,None,Pennsylvania,Allentown
4,STAFF-7,No,2,2023-01-01,None,New York,Buffalo
